# atmodeller

## Tutorial 3b: Monte Carlo experiment

We can devise a simple Monte Carlo (MC) approach to sample the probable atmospheres that can arise for different planetary conditions.

In [1]:
from atmodeller import debug_logger
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.constraints import MassConstraint, IronWustiteBufferConstraintHirschmann, SystemConstraints
from atmodeller.interfaces import GasSpecies
from atmodeller.solubilities import PeridotiteH2O, BasaltDixonCO2, BasaltLibourelN2
from atmodeller.utilities import earth_oceans_to_kg
from atmodeller.initial_condition import InitialConditionRegressor
import numpy as np
import logging

For production runs, make sure to set the logger to INFO or higher (i.e. WARNING, ERROR, or CRITICAL). Otherwise you will find that your MC runs slower just because of writing the output to the logger.

In [2]:
logger = debug_logger()
logger.setLevel(logging.INFO)

We now create the species that we are interested in. Note that we comment out CO2 in this case.

In [3]:
species: Species = Species()
species.append(GasSpecies(formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
#species.append(GasSpecies(formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(formula='N2', solubility=BasaltLibourelN2()))
species

[13:28:22 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2O using thermodynamic data in JANAF (JANAF name = H2O)
[13:28:22 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2 using thermodynamic data in JANAF (JANAF name = H2)
[13:28:22 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies O2 using thermodynamic data in JANAF (JANAF name = O2)
[13:28:22 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies CO using thermodynamic data in JANAF (JANAF name = CO)
[13:28:22 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies N2 using thermodynamic data in JANAF (JANAF name = N2)


Species([GasSpecies(formula='H2O', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x11efbc0d0>, name_in_dataset='H2O', _formula=Formula('H2O'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11effd550>), _output=None, solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x11efb3650>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(formula='H2', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x11efb3ed0>, name_in_dataset='H2', _formula=Formula('H2'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11f06e590>), _output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x11efb3790>, solid_melt_dis

Now create a planet. We recall that we can sample different planetary properties by updating the attributes of this object, even though in this tutorial we don't do this.

In [4]:
planet: Planet = Planet()

[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Mantle mass (kg) = 4208261222595110885130240.000000
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Mantle melt fraction = 1.000000
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Core mass fraction = 0.295335
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Planetary radius (m) = 6371000.000000
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Planetary mass (kg) = 5972000000000000327155712.000000
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Surface temperature (K) = 2000.000000
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Surface gravity (m/s^2) = 9.819973
[13:28:26 - atmodeller.interior_atmosphere - INFO     ] - Melt Composition = None


Now set up the main driver of the Monte Carlo (MC) approach. This establishes the ranges over which we sample certain properties.

In [5]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, number_of_realisations:int=100):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        number_of_realisation: Number of simulations to perform
    """

    # Parameters are normally distributed between bounds.
    number_ocean_moles = np.random.uniform(1, 10, number_of_realisations)
    ch_ratios = np.random.uniform(0.1, 1, number_of_realisations)
    fo2_shifts = np.random.uniform(-4, 4, number_of_realisations)

    # ppmw of Nitrogen in the mantle. 2.8 is the mantle value of N.
    N_ppmw = 2.8

    # The nitrogen mass is constant
    mass_N = N_ppmw * 1.0e-6 * planet.mantle_mass

    for realisation in range(number_of_realisations):

        mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
        mass_C = ch_ratios[realisation] * mass_H
        constraints = SystemConstraints([
            MassConstraint(species="H", value=mass_H),
            MassConstraint(species="C", value=mass_C),
            MassConstraint(species="N", value=mass_N),
            IronWustiteBufferConstraintHirschmann(log10_shift=fo2_shifts[realisation])
        ])

        # Extra quantities to write to the output
        # For example, it's often helpful to have the constraints expressed in a more convenient
        # form for analysis and plotting.
        extra = {'fO2_shift': fo2_shifts[realisation], 'C/H ratio':ch_ratios[realisation],
            'Number of ocean moles':number_ocean_moles[realisation]}

        interior_atmosphere.solve(constraints, extra_output=extra, factor=0.1)


We can run the MC as follows. This may take a minute or two to run.

In [6]:
interior_atmosphere: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, planet=planet)
monte_carlo(interior_atmosphere)

[13:28:35 - atmodeller.initial_condition   - INFO     ] - Creating InitialConditionConstant
[13:28:35 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[13:28:35 - atmodeller.interior_atmosphere - INFO     ] - Creating a reaction network
[13:28:35 - atmodeller.interior_atmosphere - INFO     ] - Species = ['H2O', 'H2', 'O2', 'CO', 'N2']
[13:28:35 - atmodeller.interior_atmosphere - INFO     ] - Reactions = 
{0: '1.0 H2O = 1.0 H2 + 0.5 O2'}
[13:28:35 - atmodeller.interior_atmosphere - INFO     ] - Solving system number 0
[13:28:37 - atmodeller.interior_atmosphere - INFO     ] - The solution converged.
[13:28:37 - atmodeller.interior_atmosphere - INFO     ] - InitialConditionConstant: RMSE (actual vs initial) = 1.009561953965116
[13:28:37 - atmodeller.interior_atmosphere - INFO     ] - {'CO': 235.77025255140236,
 'H2': 308.3157860325377,
 'H2O': 16.27070929555066,
 'N2': 1.05959559229644,
 'O2': 2.3211509283050356e-10}
[13:28:37 - atmodeller.interior_at

The simulation data can be exported to an Excel or a pickle file by setting the appropriate keyword argument in the output method:

In [7]:
interior_atmosphere.output(file_prefix='tutorial3b_monte_carlo', to_pickle=True)

[13:29:38 - atmodeller.output              - INFO     ] - Output written to tutorial3b_monte_carlo.pkl


{'atmosphere': [{'total_pressure': 561.4163434720192,
   'mean_molar_mass': 0.013445087003580615},
  {'total_pressure': 35.13413922182244,
   'mean_molar_mass': 0.027293578069171893},
  {'total_pressure': 185.38701284437565,
   'mean_molar_mass': 0.013339845208383385},
  {'total_pressure': 159.69514818475963,
   'mean_molar_mass': 0.0218547046601018},
  {'total_pressure': 459.2939587654929,
   'mean_molar_mass': 0.00819751650312897},
  {'total_pressure': 158.5487206695544, 'mean_molar_mass': 0.0265453777355581},
  {'total_pressure': 328.4916827312914,
   'mean_molar_mass': 0.01545969287133041},
  {'total_pressure': 17.703563425863965,
   'mean_molar_mass': 0.027720921057207488},
  {'total_pressure': 231.90417445716903,
   'mean_molar_mass': 0.024517649005177757},
  {'total_pressure': 360.27790098222886,
   'mean_molar_mass': 0.024520153673356825},
  {'total_pressure': 186.6358072701631,
   'mean_molar_mass': 0.009359062230907204},
  {'total_pressure': 68.81690302696418,
   'mean_molar_

We now want to use the simpler system solved above to inform the initial condition for a more complex system. Here, we now include CO2 as a species, although in principle we could include any number of extra species or even remove species compared to the previous species list. Also, we note that the species list does not have to be in the same order as before.

In [8]:
species: Species = Species()
species.append(GasSpecies(formula='H2O', solubility=PeridotiteH2O()))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
species.append(GasSpecies(formula='CO2', solubility=BasaltDixonCO2()))
species.append(GasSpecies(formula='N2', solubility=BasaltLibourelN2()))
species

[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2O using thermodynamic data in JANAF (JANAF name = H2O)
[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies H2 using thermodynamic data in JANAF (JANAF name = H2)
[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies O2 using thermodynamic data in JANAF (JANAF name = O2)
[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies CO using thermodynamic data in JANAF (JANAF name = CO)
[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies CO2 using thermodynamic data in JANAF (JANAF name = CO2)
[13:30:09 - atmodeller.interfaces          - INFO     ] - Creating GasSpecies N2 using thermodynamic data in JANAF (JANAF name = N2)


Species([GasSpecies(formula='H2O', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x11f12e5d0>, name_in_dataset='H2O', _formula=Formula('H2O'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11f14b490>), _output=None, solubility=<atmodeller.solubilities.PeridotiteH2O object at 0x11f12e310>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(formula='H2', thermodynamic_dataset=<atmodeller.interfaces.ThermodynamicDatasetJANAF object at 0x11eff5090>, name_in_dataset='H2', _formula=Formula('H2'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11f1ce210>), _output=None, solubility=<atmodeller.interfaces.NoSolubility object at 0x11eff4f90>, solid_melt_dis

In the new species list above, we also specify CO2.  Hence in the below we must provide a fill value to use for CO2 because the original data has no notion of CO2. In the below, we set the initial CO2 pressure to 1 bar.

In [9]:
initial_condition = InitialConditionRegressor.from_pickle('tutorial3b_monte_carlo.pkl', species=species, species_fill={'CO2':1}, fit=True, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

[13:30:12 - atmodeller.output              - INFO     ] - Output: Reading data from tutorial3b_monte_carlo.pkl
[13:30:12 - atmodeller.initial_condition   - INFO     ] - Creating InitialConditionRegressor
[13:30:12 - atmodeller.initial_condition   - INFO     ] - InitialConditionRegressor: Fit (None, None)
[13:30:12 - atmodeller.initial_condition   - INFO     ] - InitialConditionRegressor: Found constraints = ['H_mass', 'C_mass', 'N_mass', 'O2_fugacity']
[13:30:12 - atmodeller.initial_condition   - INFO     ] - InitialConditionRegressor: Found species = ['H2O', 'H2', 'O2', 'CO', 'CO2', 'N2']


In the above, fit = True, which means the trained data from the previous run (as computed from the output in the pickle file) is only used for the first fit_batch_size = 100 simulations. Subsequently the regressor will re-train itself on just the (fit_batch_size = 100) samples generated from the current model, discarding knowledge of the previous data it was trained on. Then, every partial_fit_batch_size = 100 simulations, it will update its training with the last batch of newly generated samples in order to better inform the selection of subsequent initial conditions. This is known as a dynamic or online learning approach.

It is necessary to pass the initial condition to the interior atmosphere system when it is created:

In [10]:
interior_atmosphere_ic: InteriorAtmosphereSystem = InteriorAtmosphereSystem(species=species, initial_condition=initial_condition, planet=planet)
monte_carlo(interior_atmosphere_ic, number_of_realisations=200)

[13:30:20 - atmodeller.interior_atmosphere - INFO     ] - Creating an interior-atmosphere system
[13:30:20 - atmodeller.interior_atmosphere - INFO     ] - Creating a reaction network
[13:30:20 - atmodeller.interior_atmosphere - INFO     ] - Species = ['H2O', 'H2', 'O2', 'CO', 'CO2', 'N2']
[13:30:20 - atmodeller.interior_atmosphere - INFO     ] - Reactions = 
{0: '0.5 O2 + 1.0 CO = 1.0 CO2', 1: '1.0 H2O = 1.0 H2 + 0.5 O2'}
[13:30:20 - atmodeller.interior_atmosphere - INFO     ] - Solving system number 0
[13:30:21 - atmodeller.interior_atmosphere - INFO     ] - The solution converged.
[13:30:21 - atmodeller.interior_atmosphere - INFO     ] - InitialConditionRegressor: RMSE (actual vs initial) = 0.1397946481771252
[13:30:21 - atmodeller.interior_atmosphere - INFO     ] - {'CO': 15.459861225620694,
 'CO2': 1.0136537475032763,
 'H2': 3.141347497099219,
 'H2O': 0.9390766046352297,
 'N2': 1.9143844250830895,
 'O2': 7.448211706900744e-09}
[13:30:21 - atmodeller.interior_atmosphere - INFO     ]

If you compare the log output for the two MC runs, you will see in the second MC example that the initial condition re-trained itself after 100 samples had been generated and then partially retrained itself every 50 samples. This keeps the RMSE between the initial guess and actual solution to a smaller value than simply guessing a constant initial condition. Also, fit = True allows you to train an initial condition on a similar but not identical model (for example, different solubility laws or gas equations of states), where once enough samples have been generated you would prefer to only use the new model to generate new estimates (since the behavior of the new model and the previous similar-but-not-the-same model will diverge).